# Python Interview Preparation — Comprehensive Guide

This notebook covers the most commonly asked Python interview questions with **detailed explanations, code examples, and edge cases**. Work through each section to build a solid understanding.

---

## Table of Contents

1. [Core Language](#1-core-language)
2. [Data Structures & Built-ins](#2-data-structures--built-ins)
3. [Functions & Scope](#3-functions--scope)
4. [Object-Oriented Programming](#4-object-oriented-programming)
5. [Iteration & Generators](#5-iteration--generators)
6. [Common Coding Questions](#6-common-coding-questions)
7. [Advanced / Senior-Level Topics](#7-advanced--senior-level-topics)
8. [Quick Gotchas & Rapid-Fire](#8-quick-gotchas--rapid-fire)

---

# 1. Core Language

These questions test whether you truly understand how Python works under the hood, not just the syntax.

## 1.1 Mutable vs Immutable Types

**Interview Question:** *"What is the difference between mutable and immutable types in Python? Give examples of each."*

### Key Concept

- **Immutable**: Once created, the object's value **cannot be changed in place**. Any "modification" creates a **new object**.  
  Examples: `int`, `float`, `str`, `tuple`, `frozenset`, `bytes`

- **Mutable**: The object's value **can be changed in place** without creating a new object.  
  Examples: `list`, `dict`, `set`, `bytearray`

### Why It Matters
- Mutable objects passed to functions can be **modified by the function** (side effects).
- Only **immutable** (hashable) objects can be used as dictionary keys or set members.
- Understanding mutability prevents subtle bugs with aliasing and shared references.

In [1]:
# --- Immutable Example: Strings ---
s = "hello"
print(f"Original id: {id(s)}")

s = s + " world"  # This creates a NEW string object
print(f"After concat id: {id(s)}")
# The id changed — a new object was created, the old one is garbage collected.

# You CANNOT do:
# s[0] = 'H'  # TypeError: 'str' object does not support item assignment

Original id: 2184771216176
After concat id: 2184771216496


In [ ]:
# --- Mutable Example: Lists ---
lst = [1, 2, 3]
print(f"Original id: {id(lst)}")

lst.append(4)  # Modified IN PLACE — same object
print(f"After append id: {id(lst)}")
# The id is the SAME — no new object was created.

print(f"List: {lst}")

In [2]:
# --- The Aliasing Trap (Classic Interview Follow-Up) ---
a = [1, 2, 3]
b = a          # b is NOT a copy — it's an alias (same object)

b.append(4)
print(f"a = {a}")  # a is also [1, 2, 3, 4] !
print(f"a is b: {a is b}")  # True — same object in memory

# Fix: use b = a.copy() or b = a[:] or b = list(a)

a = [1, 2, 3, 4]
a is b: True


In [4]:
# --- Immutable Tuples with Mutable Contents ---
# A tuple is immutable, but if it CONTAINS mutable objects, those can change.
t = ([1, 2], [3, 4])
# t[0] = [5, 6]   # TypeError — can't reassign tuple elements
t[0].append(99)    # But this works! The list inside is mutable.
print(f"t = {t}")  # ([1, 2, 99], [3, 4])

t = ([1, 2, 99], [3, 4])


## 1.2  `is` vs `==`

**Interview Question:** *"What is the difference between `is` and `==` in Python?"*

### Key Concept

| Operator | Checks | Under the hood |
|----------|--------|----------------|
| `==`     | **Value equality** — do the objects have the same value? | Calls `__eq__()` |
| `is`     | **Identity** — are they the **exact same object** in memory? | Compares `id()` |

### When to use `is`
- Checking against **singletons**: `None`, `True`, `False`
- Never use `==` for `None` — always `x is None`

In [ ]:
a = [1, 2, 3]
b = [1, 2, 3]

print(f"a == b: {a == b}")  # True  — same VALUE
print(f"a is b: {a is b}")  # False — different OBJECTS in memory

c = a
print(f"a is c: {a is c}")  # True  — c is an alias for a

In [5]:
# --- Integer Caching (CPython implementation detail) ---
# CPython caches small integers [-5, 256] as singletons.
x = 256
y = 256
print(f"256 is 256: {x is y}")  # True (cached)

x = 257
y = 257
print(f"257 is 257: {x is y}")  # May be False (not cached)
# NOTE: behavior varies — never rely on `is` for integer comparison!

256 is 256: True
257 is 257: False


In [ ]:
# --- Correct way to check for None ---
value = None

# GOOD:
if value is None:
    print("value is None")

# BAD (works but wrong idiom — could break with custom __eq__):
# if value == None:
#     print("value is None")

## 1.3 Shallow Copy vs Deep Copy

**Interview Question:** *"Explain the difference between a shallow copy and a deep copy. When would you need each?"*

### Key Concept

| Copy Type | What it does | Nested objects |
|-----------|-------------|----------------|
| **Assignment** (`b = a`) | No copy at all — alias | Same objects |
| **Shallow copy** (`copy.copy`, `list.copy`, `[:]`) | New outer container | Inner objects are **shared** |
| **Deep copy** (`copy.deepcopy`) | New everything | Inner objects are **also copied** recursively |

In [ ]:
import copy

# --- Shallow Copy ---
original = [[1, 2], [3, 4]]
shallow = copy.copy(original)  # or original.copy() or original[:]

# The outer list is new:
print(f"original is shallow: {original is shallow}")  # False

# But the inner lists are SHARED:
print(f"original[0] is shallow[0]: {original[0] is shallow[0]}")  # True

# So modifying an inner list affects both!
shallow[0].append(99)
print(f"original: {original}")  # [[1, 2, 99], [3, 4]] — SURPRISE!

In [ ]:
# --- Deep Copy ---
original = [[1, 2], [3, 4]]
deep = copy.deepcopy(original)

# Everything is independent:
print(f"original[0] is deep[0]: {original[0] is deep[0]}")  # False

deep[0].append(99)
print(f"original: {original}")  # [[1, 2], [3, 4]] — unchanged
print(f"deep:     {deep}")      # [[1, 2, 99], [3, 4]]

## 1.4 The GIL (Global Interpreter Lock)

**Interview Question:** *"What is the GIL and how does it affect multithreading in Python?"*

### Key Concept

The **GIL** is a mutex in **CPython** that allows only **one thread to execute Python bytecode at a time**, even on multi-core machines.

### Implications

| Task Type | Threading helps? | Better alternative |
|-----------|-----------------|--------------------|
| **CPU-bound** (math, processing) | No — GIL blocks parallelism | `multiprocessing`, C extensions, or `concurrent.futures.ProcessPoolExecutor` |
| **I/O-bound** (network, file) | Yes — GIL is released during I/O waits | `threading`, `asyncio` |

### Common Follow-Up
- *"Does the GIL exist in all Python implementations?"* — No. **Jython** (Java) and **IronPython** (.NET) don't have a GIL. **PyPy** has one but with different characteristics.
- *"Can you avoid the GIL?"* — Use `multiprocessing` (separate processes, separate GILs), use C extensions that release the GIL (e.g., NumPy), or use `asyncio` for I/O.
- **Python 3.13+** introduced an experimental **free-threaded mode** (`--disable-gil`) that removes the GIL entirely.

In [ ]:
import threading
import time

def cpu_bound_task(n):
    """Simulate CPU-bound work."""
    total = 0
    for i in range(n):
        total += i * i
    return total

N = 5_000_000

# Single-threaded
start = time.time()
cpu_bound_task(N)
cpu_bound_task(N)
single = time.time() - start

# Multi-threaded (GIL prevents true parallelism here)
start = time.time()
t1 = threading.Thread(target=cpu_bound_task, args=(N,))
t2 = threading.Thread(target=cpu_bound_task, args=(N,))
t1.start(); t2.start()
t1.join(); t2.join()
multi = time.time() - start

print(f"Single-threaded: {single:.3f}s")
print(f"Multi-threaded:  {multi:.3f}s")
print(f"Speedup: {single/multi:.2f}x")  # ~1.0x — no speedup due to GIL!

## 1.5 `*args` and `**kwargs`

**Interview Question:** *"What are `*args` and `**kwargs`? How do they work?"*

### Key Concept

- `*args` — Collects **extra positional** arguments into a **tuple**.
- `**kwargs` — Collects **extra keyword** arguments into a **dict**.
- The names `args`/`kwargs` are convention, not required. The `*` and `**` are what matter.

### Argument Order Rule
```python
def f(positional, *args, keyword_only, **kwargs):
    ...
```
Order: positional → `*args` → keyword-only → `**kwargs`

In [6]:
def showcase(*args, **kwargs):
    print(f"args   = {args}   (type: {type(args).__name__})")
    print(f"kwargs = {kwargs} (type: {type(kwargs).__name__})")

showcase(1, 2, 3, name="Alice", age=30)
# args   = (1, 2, 3)                  (type: tuple)
# kwargs = {'name': 'Alice', 'age': 30} (type: dict)

args   = (1, 2, 3)   (type: tuple)
kwargs = {'name': 'Alice', 'age': 30} (type: dict)


In [7]:
# --- Unpacking (the reverse operation) ---
def add(a, b, c):
    return a + b + c

nums = [1, 2, 3]
print(add(*nums))  # Unpacks list into positional args → add(1, 2, 3)

config = {'a': 10, 'b': 20, 'c': 30}
print(add(**config))  # Unpacks dict into keyword args → add(a=10, b=20, c=30)

6
60


In [8]:
# --- Practical Use Case: Decorator that wraps any function ---
import functools

def log_calls(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__}({args}, {kwargs})")
        result = func(*args, **kwargs)  # Forward ALL arguments
        print(f"  → returned {result}")
        return result
    return wrapper

@log_calls
def multiply(x, y):
    return x * y

multiply(3, 4)

Calling multiply((3, 4), {})
  → returned 12


12

---

# 2. Data Structures & Built-ins

## 2.1 List vs Tuple vs Set vs Dict

**Interview Question:** *"When would you use a list vs a tuple vs a set vs a dict?"*

| Structure | Ordered? | Mutable? | Duplicates? | Lookup | Use Case |
|-----------|----------|----------|-------------|--------|----------|
| `list`    | Yes | Yes | Yes | O(n) | General-purpose ordered collection |
| `tuple`   | Yes | No  | Yes | O(n) | Immutable records, dict keys, function returns |
| `set`     | No  | Yes | No  | **O(1)** | Membership testing, removing duplicates |
| `dict`    | Yes (3.7+) | Yes | Keys: No | **O(1)** | Key-value mapping |

### Time Complexity Cheat Sheet

| Operation | list | dict/set |
|-----------|------|----------|
| Access by index | O(1) | N/A |
| Search (`in`) | **O(n)** | **O(1)** |
| Append/Add | O(1) amortized | O(1) amortized |
| Insert at beginning | **O(n)** | O(1) |
| Delete by value | O(n) | O(1) |

In [ ]:
# --- Performance difference: list vs set for membership testing ---
import time

data_list = list(range(1_000_000))
data_set  = set(range(1_000_000))

target = 999_999  # worst case for list

start = time.perf_counter()
for _ in range(100):
    _ = target in data_list
list_time = time.perf_counter() - start

start = time.perf_counter()
for _ in range(100):
    _ = target in data_set
set_time = time.perf_counter() - start

print(f"List lookup:  {list_time:.4f}s")
print(f"Set lookup:   {set_time:.6f}s")
print(f"Set is {list_time/set_time:.0f}x faster!")

## 2.2 List Comprehensions vs Generator Expressions

**Interview Question:** *"What's the difference between `[x for x in range(n)]` and `(x for x in range(n))`?"*

| Feature | List Comprehension `[...]` | Generator Expression `(...)` |
|---------|---------------------------|-----------------------------|
| Returns | `list` (all values in memory) | `generator` (lazy, one at a time) |
| Memory | O(n) | **O(1)** |
| Reusable? | Yes | No (exhausted after one pass) |
| Speed | Slightly faster for small data | Better for large/infinite data |

In [ ]:
import sys

# List comprehension: stores all values in memory
list_comp = [x * x for x in range(1_000_000)]

# Generator expression: produces values on demand
gen_expr = (x * x for x in range(1_000_000))

print(f"List size in memory: {sys.getsizeof(list_comp):,} bytes")  # ~8 MB
print(f"Generator size:      {sys.getsizeof(gen_expr):,} bytes")   # ~200 bytes!
print(f"Type: {type(gen_expr)}")

In [ ]:
# --- Generator is exhausted after one pass ---
gen = (x for x in range(5))
print(list(gen))  # [0, 1, 2, 3, 4]
print(list(gen))  # [] — already exhausted!

In [ ]:
# --- Practical use: pass generator directly to functions ---
# No intermediate list needed!
total = sum(x * x for x in range(1_000_000))  # Memory efficient
print(f"Sum of squares: {total:,}")

# Also works with any(), all(), min(), max(), etc.
has_negative = any(x < 0 for x in [1, -2, 3])
print(f"Has negative: {has_negative}")

## 2.3 Dictionary Internals

**Interview Question:** *"How are Python dictionaries implemented? Why must keys be hashable?"*

### Key Concept

Python dicts use a **hash table**:
1. When you do `d[key] = value`, Python calls `hash(key)` to compute an integer.
2. The hash is used to find a **slot** in an internal array.
3. If two keys hash to the same slot (**collision**), Python uses **open addressing** (probing) to find the next available slot.

### Why Keys Must Be Hashable
- The key's hash must **not change** over its lifetime (otherwise lookups would fail).
- Mutable objects (`list`, `dict`, `set`) can change → their hash would change → **not allowed as keys**.
- Immutable objects (`int`, `str`, `tuple` of immutables) have stable hashes → **valid keys**.

### Since Python 3.7
- Dicts **preserve insertion order** (this was an implementation detail in 3.6, guaranteed in 3.7+).

In [ ]:
# --- Hashability ---
print(hash(42))            # Works — int is immutable
print(hash("hello"))       # Works — str is immutable
print(hash((1, 2, 3)))     # Works — tuple of immutables

try:
    hash([1, 2, 3])        # Fails! — list is mutable
except TypeError as e:
    print(f"Error: {e}")

try:
    hash({1, 2, 3})        # Fails! — set is mutable
except TypeError as e:
    print(f"Error: {e}")

# Use frozenset for hashable sets
print(hash(frozenset({1, 2, 3})))  # Works!

In [ ]:
# --- Tuple as dict key (common pattern) ---
# Great for coordinate mappings, caching, etc.
grid = {}
grid[(0, 0)] = "origin"
grid[(1, 2)] = "point A"
print(grid[(0, 0)])  # "origin"

# But a tuple containing a list is NOT hashable:
try:
    hash(([1, 2], [3, 4]))  # Fails!
except TypeError as e:
    print(f"Error: {e}")

## 2.4 `defaultdict`, `Counter`, `OrderedDict`

**Interview Question:** *"What are `defaultdict` and `Counter`? When would you use them?"*

In [ ]:
from collections import defaultdict, Counter, OrderedDict

# --- defaultdict: auto-creates missing keys ---
# Without defaultdict:
word_groups = {}
words = ["apple", "banana", "avocado", "blueberry", "apricot"]
for w in words:
    first = w[0]
    if first not in word_groups:  # Manual check needed
        word_groups[first] = []
    word_groups[first].append(w)

# With defaultdict (much cleaner):
word_groups = defaultdict(list)  # Missing keys auto-create empty list
for w in words:
    word_groups[w[0]].append(w)  # No KeyError ever!

print(dict(word_groups))

In [ ]:
# --- Counter: counting made easy ---
text = "abracadabra"
counter = Counter(text)
print(f"Counts: {counter}")
print(f"Most common 3: {counter.most_common(3)}")

# Counter arithmetic
c1 = Counter("aabbb")
c2 = Counter("abbcc")
print(f"c1 + c2 = {c1 + c2}")  # Adds counts
print(f"c1 - c2 = {c1 - c2}")  # Subtracts (drops zero/negative)

In [ ]:
# --- OrderedDict: when insertion order comparison matters ---
# Since Python 3.7, regular dicts preserve order, but:
# - Regular dict: order is NOT considered in equality
# - OrderedDict: order IS considered in equality

d1 = {'a': 1, 'b': 2}
d2 = {'b': 2, 'a': 1}
print(f"dict == dict: {d1 == d2}")  # True (order ignored)

od1 = OrderedDict([('a', 1), ('b', 2)])
od2 = OrderedDict([('b', 2), ('a', 1)])
print(f"OrderedDict == OrderedDict: {od1 == od2}")  # False (order matters!)

---

# 3. Functions & Scope

## 3.1 LEGB Rule (Scope Resolution)

**Interview Question:** *"How does Python resolve variable names? What is the LEGB rule?"*

Python looks up names in this order:

1. **L**ocal — Inside the current function
2. **E**nclosing — In any enclosing function (closures)
3. **G**lobal — At the module level
4. **B**uilt-in — In Python's built-in namespace (`len`, `print`, etc.)

In [ ]:
x = "global"  # G

def outer():
    x = "enclosing"  # E
    
    def inner():
        x = "local"  # L
        print(f"inner sees: {x}")  # "local"
    
    inner()
    print(f"outer sees: {x}")  # "enclosing"

outer()
print(f"module sees: {x}")  # "global"

In [ ]:
# --- `global` and `nonlocal` keywords ---
count = 0

def increment_global():
    global count  # Without this, assigning count would create a LOCAL variable
    count += 1

increment_global()
increment_global()
print(f"Global count: {count}")  # 2

def outer_counter():
    count = 0
    
    def increment():
        nonlocal count  # Modify the ENCLOSING variable, not create a local one
        count += 1
        return count
    
    return increment

counter = outer_counter()
print(counter())  # 1
print(counter())  # 2

## 3.2 Closures

**Interview Question:** *"What is a closure in Python? Can you give an example?"*

### Key Concept

A **closure** is a function that **remembers the variables from its enclosing scope**, even after the outer function has finished executing.

Three conditions for a closure:
1. A nested (inner) function
2. The inner function references variables from the enclosing scope
3. The outer function returns the inner function

In [ ]:
def make_multiplier(factor):
    """Factory function that creates multiplier functions."""
    def multiply(x):
        return x * factor  # 'factor' is captured from enclosing scope
    return multiply

double = make_multiplier(2)
triple = make_multiplier(3)

print(double(5))  # 10
print(triple(5))  # 15

# The closure stores the enclosing variables:
print(double.__closure__[0].cell_contents)  # 2

In [ ]:
# --- Classic Closure Pitfall: Late Binding ---
# This is a VERY common interview question!

# BUG: All functions capture the SAME variable 'i', which ends at 4
functions_bad = []
for i in range(5):
    functions_bad.append(lambda: i)  # All lambdas see the SAME 'i'

print([f() for f in functions_bad])  # [4, 4, 4, 4, 4] — NOT [0, 1, 2, 3, 4]!

# FIX: Use default argument to capture value at creation time
functions_good = []
for i in range(5):
    functions_good.append(lambda i=i: i)  # Default arg captures current value

print([f() for f in functions_good])  # [0, 1, 2, 3, 4] — correct!

## 3.3 Decorators

**Interview Question:** *"What is a decorator? Write one from scratch."*

### Key Concept

A decorator is a function that **takes a function as input** and **returns a new function** that usually extends the original's behavior. The `@decorator` syntax is syntactic sugar:

```python
@decorator
def func(): ...

# is exactly the same as:
func = decorator(func)
```

In [ ]:
import functools
import time

# --- Basic Decorator: Timer ---
def timer(func):
    @functools.wraps(func)  # Preserves __name__, __doc__, etc.
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - start
        print(f"{func.__name__} took {elapsed:.4f}s")
        return result
    return wrapper

@timer
def slow_function(n):
    """Sum numbers slowly."""
    return sum(range(n))

result = slow_function(1_000_000)
print(f"Result: {result}")
print(f"Name preserved: {slow_function.__name__}")  # "slow_function" (thanks to @wraps)
print(f"Doc preserved:  {slow_function.__doc__}")    # "Sum numbers slowly."

In [14]:
import functools
import time

def timer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - start
        print(f"{func.__name__} took {elapsed:.4f} seconds to run")
        return result
    return wrapper

@timer
def slow(n):
    return sum(range(n))
result = slow(1_000_000)

slow took 0.0227 seconds to run


In [15]:
# --- Decorator WITH Arguments ---
# Requires an extra layer of nesting.

def retry(max_attempts=3):
    """Decorator that retries a function on failure."""
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(1, max_attempts + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"  Attempt {attempt}/{max_attempts} failed: {e}")
                    if attempt == max_attempts:
                        raise
        return wrapper
    return decorator

call_count = 0

@retry(max_attempts=3)
def flaky_function():
    """Simulates a function that fails sometimes."""
    global call_count
    call_count += 1
    if call_count < 3:
        raise ConnectionError("Server down")
    return "Success!"

print(flaky_function())  # Retries twice, succeeds on third

  Attempt 1/3 failed: Server down
  Attempt 2/3 failed: Server down
Success!


In [18]:
# --- Why @functools.wraps matters ---
def bad_decorator(func):
    # @functools.wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper  # No @wraps!

@bad_decorator
def my_func():
    """My docstring."""
    pass

print(f"Without @wraps: name={my_func.__name__}, doc={my_func.__doc__}")
# name=wrapper, doc=None — WRONG! The function's identity is lost.

Without @wraps: name=wrapper, doc=None


## 3.4 Mutable Default Arguments

**Interview Question:** *"What's wrong with using a mutable default argument like `def f(lst=[])`?"*

### The Bug
Default arguments are evaluated **once** when the function is **defined**, not each time it's called. So a mutable default is **shared across all calls**.

In [ ]:
# --- The Bug ---
def append_to_bad(item, lst=[]):
    lst.append(item)
    return lst

print(append_to_bad(1))  # [1]
print(append_to_bad(2))  # [1, 2]  — BUG! Expected [2]
print(append_to_bad(3))  # [1, 2, 3] — The list persists!

# You can see the shared default:
print(f"Default: {append_to_bad.__defaults__}")  # ([1, 2, 3],)

In [ ]:
# --- The Fix: Use None as sentinel ---
def append_to_good(item, lst=None):
    if lst is None:
        lst = []  # Fresh list created on EACH call
    lst.append(item)
    return lst

print(append_to_good(1))  # [1]
print(append_to_good(2))  # [2] — Correct!

## 3.5 First-Class Functions

**Interview Question:** *"What does it mean that functions are first-class objects in Python?"*

Functions can be:
- Assigned to variables
- Passed as arguments to other functions
- Returned from functions
- Stored in data structures

In [ ]:
# --- Functions as arguments ---
def apply_operation(func, x, y):
    return func(x, y)

def add(a, b): return a + b
def mul(a, b): return a * b

print(apply_operation(add, 3, 4))  # 7
print(apply_operation(mul, 3, 4))  # 12

# --- map, filter, lambda ---
numbers = [1, 2, 3, 4, 5]

squared = list(map(lambda x: x**2, numbers))
print(f"Squared: {squared}")

evens = list(filter(lambda x: x % 2 == 0, numbers))
print(f"Evens: {evens}")

# Prefer list comprehensions for readability:
squared = [x**2 for x in numbers]
evens = [x for x in numbers if x % 2 == 0]

---

# 4. Object-Oriented Programming

## 4.1 `__init__` vs `__new__`

**Interview Question:** *"What's the difference between `__new__` and `__init__`?"*

| Method | Purpose | When called | Returns |
|--------|---------|-------------|--------|
| `__new__` | **Creates** the instance (allocates memory) | Before `__init__` | The new instance |
| `__init__` | **Initializes** the instance (sets attributes) | After `__new__` | `None` |

You rarely override `__new__`. The main use cases:
- Subclassing **immutable types** (`int`, `str`, `tuple`)
- Implementing the **Singleton pattern**

In [ ]:
# --- Normal class: just use __init__ ---
class Point:
    def __init__(self, x, y):
        self.x = x  # Initialize attributes
        self.y = y
    
    def __repr__(self):
        return f"Point({self.x}, {self.y})"

p = Point(3, 4)
print(p)

In [ ]:
# --- Singleton Pattern using __new__ ---
class Singleton:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self):
        self.value = 42

a = Singleton()
b = Singleton()
print(f"a is b: {a is b}")  # True — same object!

In [ ]:
# --- Subclassing immutable type with __new__ ---
class UpperStr(str):
    """A string that's always uppercase."""
    def __new__(cls, value):
        # Must modify during creation since str is immutable
        instance = super().__new__(cls, value.upper())
        return instance

s = UpperStr("hello")
print(s)            # HELLO
print(type(s))      # <class '__main__.UpperStr'>
print(isinstance(s, str))  # True

## 4.2 MRO (Method Resolution Order)

**Interview Question:** *"How does Python resolve methods in multiple inheritance (diamond problem)?"*

### Key Concept

Python uses the **C3 Linearization** algorithm to determine the order in which classes are searched for a method. You can inspect it with `ClassName.__mro__` or `ClassName.mro()`.

The rule (simplified): **children before parents, left before right, and each class appears only once**.

In [ ]:
# --- Diamond Inheritance ---
#       A
#      / \
#     B   C
#      \ /
#       D

class A:
    def who(self): return "A"

class B(A):
    def who(self): return "B"

class C(A):
    def who(self): return "C"

class D(B, C):  # Left-to-right: B before C
    pass

d = D()
print(f"d.who() = {d.who()}")  # "B" — B comes before C in MRO

# Inspect the MRO:
print(f"MRO: {[cls.__name__ for cls in D.__mro__]}")
# ['D', 'B', 'C', 'A', 'object']

In [ ]:
# --- super() follows the MRO, NOT the parent class ---
class A:
    def greet(self):
        print("A.greet")

class B(A):
    def greet(self):
        print("B.greet")
        super().greet()  # Calls next in MRO, not necessarily A!

class C(A):
    def greet(self):
        print("C.greet")
        super().greet()

class D(B, C):
    def greet(self):
        print("D.greet")
        super().greet()

print("Call chain:")
D().greet()
# D.greet → B.greet → C.greet → A.greet
# Notice: B.super() calls C (next in MRO), NOT A!

## 4.3 `@staticmethod` vs `@classmethod` vs Instance Method

**Interview Question:** *"What's the difference between a static method, class method, and instance method?"*

| Type | First Arg | Access to | Decorator |
|------|-----------|-----------|----------|
| Instance method | `self` (instance) | Instance + class attributes | None |
| Class method | `cls` (class) | Class attributes only | `@classmethod` |
| Static method | Nothing | Nothing (utility function) | `@staticmethod` |

In [ ]:
class Pizza:
    base_price = 10  # Class attribute
    
    def __init__(self, toppings):
        self.toppings = toppings  # Instance attribute
    
    # Instance method: accesses self (instance) and cls (via self.__class__)
    def price(self):
        return self.base_price + len(self.toppings) * 2
    
    # Class method: alternative constructor (factory method)
    @classmethod
    def margherita(cls):
        return cls(["mozzarella", "tomato"])  # Uses cls, not Pizza directly
    
    # Static method: utility, no access to instance or class
    @staticmethod
    def validate_topping(topping):
        return topping.lower() in ["mozzarella", "tomato", "pepperoni", "mushroom"]

# Instance method
p = Pizza(["pepperoni", "mushroom"])
print(f"Price: ${p.price()}")

# Class method as factory
m = Pizza.margherita()
print(f"Margherita toppings: {m.toppings}")

# Static method
print(f"Is 'olive' valid? {Pizza.validate_topping('olive')}")

## 4.4 Dunder (Magic) Methods

**Interview Question:** *"What are dunder methods? Explain `__repr__` vs `__str__`."*

### `__repr__` vs `__str__`

| Method | Purpose | Audience | Fallback |
|--------|---------|----------|----------|
| `__repr__` | Unambiguous, developer-facing | Developers | Used if `__str__` is missing |
| `__str__` | Readable, user-facing | End users | Falls back to `__repr__` |

**Rule of thumb**: `__repr__` should ideally return a string that could **recreate the object** (e.g., `Point(3, 4)`).

In [ ]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"Vector({self.x}, {self.y})"  # Developer-facing
    
    def __str__(self):
        return f"({self.x}, {self.y})"  # User-facing
    
    def __eq__(self, other):
        return isinstance(other, Vector) and self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))  # Must define if __eq__ is defined
    
    def __len__(self):
        return 2  # A 2D vector has 2 components
    
    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)
    
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)
    
    def __abs__(self):
        return (self.x**2 + self.y**2) ** 0.5
    
    def __bool__(self):
        return abs(self) > 0  # Zero vector is falsy

v1 = Vector(3, 4)
v2 = Vector(1, 2)

print(repr(v1))       # Vector(3, 4)
print(str(v1))        # (3, 4)
print(v1 + v2)        # (4, 6)  — uses __str__ via print
print(v1 * 3)         # (9, 12)
print(abs(v1))        # 5.0
print(len(v1))        # 2
print(bool(Vector(0, 0)))  # False

## 4.5 `@property` — Pythonic Getters/Setters

**Interview Question:** *"How do you implement getters and setters in Python?"*

Use `@property` instead of Java-style `get_x()` / `set_x()` methods.

In [ ]:
class Temperature:
    def __init__(self, celsius):
        self.celsius = celsius  # This triggers the setter!
    
    @property
    def celsius(self):
        """Get temperature in Celsius."""
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        """Set temperature with validation."""
        if value < -273.15:
            raise ValueError("Temperature below absolute zero!")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        """Computed property — read-only."""
        return self._celsius * 9/5 + 32

t = Temperature(100)
print(f"{t.celsius}°C = {t.fahrenheit}°F")  # 100°C = 212.0°F

t.celsius = 0  # Uses the setter (with validation)
print(f"{t.celsius}°C = {t.fahrenheit}°F")  # 0°C = 32.0°F

try:
    t.celsius = -300  # Below absolute zero!
except ValueError as e:
    print(f"Error: {e}")

---

# 5. Iteration & Generators

## 5.1 Iterators vs Iterables

**Interview Question:** *"What's the difference between an iterable and an iterator?"*

| Concept | Has method | Returns |
|---------|-----------|--------|
| **Iterable** | `__iter__()` | An **iterator** |
| **Iterator** | `__iter__()` AND `__next__()` | Next value, or raises `StopIteration` |

- Every **iterator** is an iterable (it has `__iter__` that returns `self`).
- Not every **iterable** is an iterator (e.g., `list` is iterable but not an iterator).
- `for x in obj:` calls `iter(obj)` to get an iterator, then calls `next()` repeatedly.

In [ ]:
# --- How a for loop actually works ---
my_list = [10, 20, 30]

# This:
for x in my_list:
    print(x, end=" ")
print()

# Is equivalent to:
iterator = iter(my_list)  # Calls my_list.__iter__()
while True:
    try:
        x = next(iterator)  # Calls iterator.__next__()
        print(x, end=" ")
    except StopIteration:
        break
print()

In [ ]:
# --- Custom Iterator: Countdown ---
class Countdown:
    def __init__(self, start):
        self.current = start
    
    def __iter__(self):
        return self  # The iterator is the object itself
    
    def __next__(self):
        if self.current <= 0:
            raise StopIteration
        value = self.current
        self.current -= 1
        return value

for num in Countdown(5):
    print(num, end=" ")  # 5 4 3 2 1
print()

## 5.2 Generators & `yield`

**Interview Question:** *"What is a generator? How does `yield` work?"*

### Key Concept

A **generator** is a function that uses `yield` instead of `return`. When called, it returns a **generator object** (an iterator) without executing any code. Each call to `next()` runs the function **until the next `yield`**, pauses, and returns the yielded value.

### Why Use Generators?
- **Memory efficient**: Values are produced one at a time (lazy evaluation)
- **Can represent infinite sequences**
- **Simpler than writing a full iterator class**

In [ ]:
def fibonacci():
    """Infinite Fibonacci generator."""
    a, b = 0, 1
    while True:  # Infinite! But no memory issues.
        yield a
        a, b = b, a + b

# Take only the first 10 values
fib = fibonacci()
first_10 = [next(fib) for _ in range(10)]
print(f"First 10 Fibonacci: {first_10}")

In [ ]:
# --- Generator vs List: Memory comparison ---
import sys

def gen_squares(n):
    for i in range(n):
        yield i * i

# Generator: O(1) memory regardless of n
gen = gen_squares(1_000_000)
print(f"Generator: {sys.getsizeof(gen)} bytes")

# List: O(n) memory
lst = [i * i for i in range(1_000_000)]
print(f"List: {sys.getsizeof(lst):,} bytes")

In [ ]:
# --- How yield pauses and resumes execution ---
def demo_generator():
    print("Step 1")
    yield "first"
    print("Step 2")
    yield "second"
    print("Step 3 (cleanup)")
    # No more yields → StopIteration will be raised

gen = demo_generator()
print(f"Got: {next(gen)}")  # Runs until first yield
print("--- paused ---")
print(f"Got: {next(gen)}")  # Resumes from first yield to second
print("--- paused ---")
try:
    next(gen)  # Runs remaining code, then StopIteration
except StopIteration:
    print("Generator exhausted")

## 5.3 `yield from`

**Interview Question:** *"What does `yield from` do?"*

`yield from iterable` delegates iteration to another iterable/generator. It's equivalent to `for item in iterable: yield item` but also properly forwards `.send()`, `.throw()`, and `.close()`.

In [ ]:
# --- yield from: flattening nested generators ---
def flatten(nested):
    for item in nested:
        if isinstance(item, (list, tuple)):
            yield from flatten(item)  # Delegate to recursive call
        else:
            yield item

data = [1, [2, 3, [4, 5]], 6, [7, [8, 9]]]
print(list(flatten(data)))  # [1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
# --- yield from: chaining generators ---
def gen_a():
    yield 1
    yield 2

def gen_b():
    yield 3
    yield 4

def combined():
    yield from gen_a()
    yield from gen_b()

print(list(combined()))  # [1, 2, 3, 4]

---

# 6. Common Coding Questions

These are the hands-on coding problems you might be asked to solve live.

## 6.1 Reverse a String / List

In [ ]:
s = "hello world"

# Method 1: Slicing (most Pythonic)
print(s[::-1])  # "dlrow olleh"

# Method 2: reversed() built-in
print("".join(reversed(s)))  # "dlrow olleh"

# Method 3: Manual (for interview coding)
def reverse_string(s):
    chars = list(s)
    left, right = 0, len(chars) - 1
    while left < right:
        chars[left], chars[right] = chars[right], chars[left]
        left += 1
        right -= 1
    return "".join(chars)

print(reverse_string(s))

## 6.2 Find Duplicates in a List

In [ ]:
nums = [1, 3, 2, 5, 3, 1, 7, 2]

# Method 1: Set-based (O(n) time, O(n) space)
def find_duplicates(lst):
    seen = set()
    duplicates = set()
    for item in lst:
        if item in seen:
            duplicates.add(item)
        seen.add(item)
    return duplicates

print(f"Duplicates: {find_duplicates(nums)}")

# Method 2: Counter-based
from collections import Counter
dupes = [item for item, count in Counter(nums).items() if count > 1]
print(f"Duplicates: {dupes}")

# Method 3: One-liner to remove duplicates (preserving order)
unique = list(dict.fromkeys(nums))  # dict preserves insertion order
print(f"Unique (ordered): {unique}")

## 6.3 Merge Two Dictionaries

In [ ]:
d1 = {'a': 1, 'b': 2}
d2 = {'b': 3, 'c': 4}  # Note: 'b' exists in both

# Method 1: | operator (Python 3.9+, most Pythonic)
merged = d1 | d2  # d2 values win on conflicts
print(f"d1 | d2 = {merged}")  # {'a': 1, 'b': 3, 'c': 4}

# Method 2: Unpacking (Python 3.5+)
merged = {**d1, **d2}  # d2 values win on conflicts
print(f"{{**d1, **d2}} = {merged}")

# Method 3: update() (modifies in place)
merged = d1.copy()
merged.update(d2)
print(f"update = {merged}")

# Method 4: |= operator (in-place merge, Python 3.9+)
d1_copy = d1.copy()
d1_copy |= d2
print(f"|= {d1_copy}")

## 6.4 Flatten a Nested List

In [ ]:
nested = [1, [2, 3, [4, 5]], 6, [[7], 8]]

# Method 1: Recursive generator (handles arbitrary depth)
def flatten(lst):
    for item in lst:
        if isinstance(item, list):
            yield from flatten(item)
        else:
            yield item

print(list(flatten(nested)))  # [1, 2, 3, 4, 5, 6, 7, 8]

# Method 2: Iterative with stack
def flatten_iterative(lst):
    stack = lst[::-1]  # Reverse so we pop from the left
    result = []
    while stack:
        item = stack.pop()
        if isinstance(item, list):
            stack.extend(item[::-1])
        else:
            result.append(item)
    return result

print(flatten_iterative(nested))  # [1, 2, 3, 4, 5, 6, 7, 8]

# Method 3: One level only — use itertools.chain
import itertools
shallow_nested = [[1, 2], [3, 4], [5, 6]]
print(list(itertools.chain.from_iterable(shallow_nested)))  # [1, 2, 3, 4, 5, 6]

## 6.5 LRU Cache

In [ ]:
# --- Method 1: functools.lru_cache (use this in practice) ---
from functools import lru_cache

@lru_cache(maxsize=128)
def expensive_computation(n):
    """Simulate expensive work."""
    print(f"  Computing for n={n}...")
    return sum(i * i for i in range(n))

print(expensive_computation(1000))  # Computes
print(expensive_computation(1000))  # Cached! No "Computing" message
print(expensive_computation(500))   # Computes (different arg)

print(f"Cache info: {expensive_computation.cache_info()}")

In [ ]:
# --- Method 2: Implement LRU Cache from scratch (interview classic) ---
from collections import OrderedDict

class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = OrderedDict()
    
    def get(self, key):
        if key not in self.cache:
            return -1
        self.cache.move_to_end(key)  # Mark as recently used
        return self.cache[key]
    
    def put(self, key, value):
        if key in self.cache:
            self.cache.move_to_end(key)
        self.cache[key] = value
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)  # Remove LEAST recently used

cache = LRUCache(2)
cache.put(1, "a")
cache.put(2, "b")
print(cache.get(1))    # "a" — marks 1 as recently used
cache.put(3, "c")      # Evicts key 2 (least recently used)
print(cache.get(2))    # -1 — evicted!
print(cache.get(3))    # "c"

## 6.6 Count Word Frequency

In [ ]:
text = "the quick brown fox jumps over the lazy dog the fox"

# Method 1: Counter (best)
from collections import Counter
word_freq = Counter(text.split())
print(f"Word frequencies: {word_freq}")
print(f"Top 3: {word_freq.most_common(3)}")

# Method 2: Manual with dict
freq = {}
for word in text.split():
    freq[word] = freq.get(word, 0) + 1  # .get() with default
print(f"Manual: {freq}")

# Method 3: defaultdict
from collections import defaultdict
freq = defaultdict(int)
for word in text.split():
    freq[word] += 1
print(f"defaultdict: {dict(freq)}")

---

# 7. Advanced / Senior-Level Topics

## 7.1 Context Managers

**Interview Question:** *"What is a context manager? How do you create one?"*

### Key Concept

A context manager ensures that **setup and cleanup code** always runs, even if exceptions occur. The `with` statement uses the `__enter__` and `__exit__` protocol.

```python
with open('file.txt') as f:  # __enter__ opens file
    data = f.read()
# __exit__ closes file (even if an exception was raised)
```

In [ ]:
# --- Method 1: Class-based context manager ---
class Timer:
    def __enter__(self):
        import time
        self.start = time.perf_counter()
        return self  # This is what 'as' binds to
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        import time
        self.elapsed = time.perf_counter() - self.start
        print(f"Elapsed: {self.elapsed:.4f}s")
        return False  # Don't suppress exceptions

with Timer() as t:
    total = sum(range(1_000_000))
    print(f"Sum: {total}")

In [ ]:
# --- Method 2: Generator-based with contextlib (simpler!) ---
from contextlib import contextmanager
import time

@contextmanager
def timer(label="Block"):
    start = time.perf_counter()
    try:
        yield  # Everything before yield is __enter__, after is __exit__
    finally:
        elapsed = time.perf_counter() - start
        print(f"{label}: {elapsed:.4f}s")

with timer("Computation"):
    total = sum(range(1_000_000))

In [ ]:
# --- Practical example: Temporary directory change ---
import os
from contextlib import contextmanager

@contextmanager
def change_dir(path):
    """Temporarily change working directory."""
    old_dir = os.getcwd()
    try:
        os.chdir(path)
        yield
    finally:
        os.chdir(old_dir)  # Always restore, even on error

print(f"Before: {os.getcwd()}")
with change_dir(".."):
    print(f"Inside: {os.getcwd()}")
print(f"After:  {os.getcwd()}")  # Restored!

## 7.2 Metaclasses

**Interview Question:** *"What is a metaclass? When would you use one?"*

### Key Concept

- A **class** is an instance of a **metaclass**.
- The default metaclass is `type`. When you write `class Foo:`, Python calls `type('Foo', (bases,), namespace)`.
- Metaclasses let you **customize class creation** itself — like a class factory.

### When to Use
- Enforcing coding conventions on all subclasses
- Auto-registering classes (plugin systems)
- ORMs (like Django models, SQLAlchemy)
- 99% of the time, you don't need metaclasses — use decorators or `__init_subclass__` instead.

In [ ]:
# --- Classes are instances of type ---
class Foo:
    pass

print(type(Foo))       # <class 'type'> — Foo is an instance of type
print(type(type))      # <class 'type'> — type is its own metaclass!
print(isinstance(Foo, type))  # True

In [ ]:
# --- Custom Metaclass: Auto-registry ---
class PluginMeta(type):
    registry = []
    
    def __new__(mcs, name, bases, namespace):
        cls = super().__new__(mcs, name, bases, namespace)
        if bases:  # Don't register the base class itself
            PluginMeta.registry.append(cls)
        return cls

class Plugin(metaclass=PluginMeta):
    pass

class AudioPlugin(Plugin):
    pass

class VideoPlugin(Plugin):
    pass

print(f"Registered plugins: {[c.__name__ for c in PluginMeta.registry]}")
# ['AudioPlugin', 'VideoPlugin']

In [ ]:
# --- Modern Alternative: __init_subclass__ (Python 3.6+) ---
# Simpler and preferred over metaclasses for most use cases.

class Plugin:
    _registry = []
    
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        Plugin._registry.append(cls)

class AudioPlugin(Plugin):
    pass

class VideoPlugin(Plugin):
    pass

print(f"Registered: {[c.__name__ for c in Plugin._registry]}")

## 7.3 `asyncio` — Asynchronous Programming

**Interview Question:** *"What is `asyncio`? How is it different from threading?"*

### Key Concept

| Feature | `threading` | `asyncio` |
|---------|------------|----------|
| Concurrency model | **Preemptive** (OS switches threads) | **Cooperative** (you decide when to switch with `await`) |
| Overhead | Higher (OS threads) | Lower (single thread, event loop) |
| Race conditions | Yes (need locks) | Rare (single thread) |
| Best for | CPU-bound + I/O | Many I/O operations (web servers, APIs) |

- `async def` defines a **coroutine**.
- `await` pauses the coroutine and lets the event loop run other tasks.
- `asyncio.gather()` runs multiple coroutines concurrently.

In [ ]:
import asyncio
import time

async def fetch_data(name, delay):
    """Simulate an I/O operation (API call, DB query, etc.)."""
    print(f"  {name}: starting (will take {delay}s)")
    await asyncio.sleep(delay)  # Non-blocking sleep
    print(f"  {name}: done!")
    return f"{name}_result"

async def main():
    start = time.time()
    
    # Run 3 tasks CONCURRENTLY (not sequentially!)
    results = await asyncio.gather(
        fetch_data("API", 2),
        fetch_data("DB", 1),
        fetch_data("Cache", 0.5),
    )
    
    elapsed = time.time() - start
    print(f"\nAll done in {elapsed:.1f}s (not 3.5s!)")
    print(f"Results: {results}")

# In Jupyter, use await directly:
await main()
# In scripts, use: asyncio.run(main())

## 7.4 Type Hints

**Interview Question:** *"What are type hints in Python? Are they enforced at runtime?"*

### Key Concept

- Type hints are **annotations** that describe expected types.
- They are **NOT enforced at runtime** — Python is still dynamically typed.
- They're used by **static analysis tools** (`mypy`, `pyright`), IDEs, and documentation.
- Added in Python 3.5 via PEP 484.

In [ ]:
from typing import Optional, Union, Protocol

# --- Basic type hints ---
def greet(name: str, times: int = 1) -> str:
    return (f"Hello, {name}! ") * times

print(greet("Alice", 2))

# This runs fine — no runtime enforcement!
print(greet(42, "oops"))  # Works but mypy would catch it

In [ ]:
# --- Common type hint patterns ---
from typing import Optional, Union

# Optional: can be None
def find_user(user_id: int) -> Optional[str]:  # Same as str | None
    if user_id == 1:
        return "Alice"
    return None

# Union: multiple types (Python 3.10+ can use X | Y)
def process(data: Union[str, list]) -> int:  # str | list in 3.10+
    return len(data)

# Generic collections (Python 3.9+ can use built-in types)
def first_element(items: list[int]) -> int | None:  # 3.10+ syntax
    return items[0] if items else None

print(find_user(1))
print(process([1, 2, 3]))
print(first_element([10, 20]))

In [ ]:
# --- Protocol: Structural subtyping (duck typing with type safety) ---
from typing import Protocol

class Drawable(Protocol):
    def draw(self) -> str: ...

class Circle:
    def draw(self) -> str:
        return "Drawing circle"

class Square:
    def draw(self) -> str:
        return "Drawing square"

# No inheritance needed! Just matching the interface.
def render(shape: Drawable) -> None:
    print(shape.draw())

render(Circle())  # Works — Circle has .draw()
render(Square())  # Works — Square has .draw()

## 7.5 Memory Management

**Interview Question:** *"How does Python manage memory? What is garbage collection?"*

### Key Concept

Python uses **two mechanisms**:

1. **Reference counting** (primary): Each object tracks how many references point to it. When the count drops to 0, the object is immediately freed.

2. **Cyclic garbage collector** (secondary): Handles reference cycles (A → B → A) that reference counting can't detect. Runs periodically on "generations" (gen 0, 1, 2).

In [ ]:
import sys
import gc

# --- Reference counting ---
a = [1, 2, 3]  # refcount = 1
b = a           # refcount = 2
print(f"Refcount of a: {sys.getrefcount(a)}")  # +1 for the getrefcount arg itself

del b           # refcount back to 1
print(f"After del b:   {sys.getrefcount(a)}")

In [ ]:
# --- Reference cycle (needs GC) ---
class Node:
    def __init__(self, name):
        self.name = name
        self.ref = None
    def __del__(self):
        print(f"  Deleting {self.name}")

# Create a cycle: a → b → a
a = Node("A")
b = Node("B")
a.ref = b
b.ref = a

# Delete our references
del a
del b
# Objects still exist! (cycle keeps refcount > 0)

# Force garbage collection
print("Running GC:")
gc.collect()  # Now the cyclic GC detects and cleans up the cycle

# GC stats
print(f"\nGC thresholds: {gc.get_threshold()}")  # (700, 10, 10) — gen 0, 1, 2

---

# 8. Quick Gotchas & Rapid-Fire

These are often asked as rapid-fire questions to test your intuition.

In [ ]:
# ============================================
# GOTCHA 1: Identity vs Equality for collections
# ============================================
print("--- Identity vs Equality ---")
print(f"[] == []: {[] == []}")
print(f"[] is []: {[] is []}")
print(f"() == (): {() == ()}")
print(f"() is (): {() is ()}")
# Empty tuples may be cached (implementation detail)

In [ ]:
# ============================================
# GOTCHA 2: String concatenation performance
# ============================================
import time

# BAD: O(n²) — each += creates a new string
start = time.time()
s = ""
for i in range(100_000):
    s += str(i)
bad_time = time.time() - start

# GOOD: O(n) — join is optimized
start = time.time()
s = "".join(str(i) for i in range(100_000))
good_time = time.time() - start

print(f"+=  loop: {bad_time:.4f}s")
print(f"join():   {good_time:.4f}s")
print(f"join is {bad_time/good_time:.1f}x faster")

In [ ]:
# ============================================
# GOTCHA 3: Shared reference with x = y = []
# ============================================
print("--- Shared Reference ---")
x = y = []  # SAME object!
x.append(1)
print(f"y = {y}")     # [1] — surprise!
print(f"x is y: {x is y}")  # True

# Fix:
x, y = [], []  # Two DIFFERENT objects
x.append(1)
print(f"y = {y}")     # [] — independent

In [ ]:
# ============================================
# GOTCHA 4: Single-element tuple
# ============================================
print("--- Tuple Gotcha ---")
not_a_tuple = (1)     # This is just an int!
a_tuple = (1,)        # Need the trailing comma!
also_tuple = 1,       # Parentheses are optional

print(f"type((1)):  {type(not_a_tuple)}")
print(f"type((1,)): {type(a_tuple)}")
print(f"type(1,):   {type(also_tuple)}")

In [ ]:
# ============================================
# GOTCHA 5: Chained comparisons
# ============================================
print("--- Chained Comparisons ---")
x = 5
print(f"1 < x < 10:  {1 < x < 10}")    # True — Pythonic!
print(f"10 > x > 1:  {10 > x > 1}")    # True
print(f"1 < x > 3:   {1 < x > 3}")     # True (both conditions)

# Surprising:
print(f"(False == False) in [False]: {(False == False) in [False]}")  # True? False?
# False == False is True, and True is NOT in [False], so... False!

In [ ]:
# ============================================
# GOTCHA 6: Boolean is a subclass of int
# ============================================
print("--- Bool is Int ---")
print(f"isinstance(True, int): {isinstance(True, int)}")  # True!
print(f"True + True: {True + True}")    # 2
print(f"True * 10:   {True * 10}")      # 10
print(f"sum([True, False, True]): {sum([True, False, True])}")  # 2 (counts Trues!)

In [ ]:
# ============================================
# GOTCHA 7: Walrus operator (:=) — Python 3.8+
# ============================================
print("--- Walrus Operator ---")

# Without walrus: compute twice or use temp variable
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# With walrus: assign and test in one expression
if (n := len(data)) > 5:
    print(f"List is long ({n} elements)")

# Useful in while loops:
import io
buffer = io.StringIO("line1\nline2\nline3\n")
while (line := buffer.readline().strip()):
    print(f"  Read: {line}")

In [ ]:
# ============================================
# GOTCHA 8: Dictionary/set comprehensions
# ============================================
print("--- Comprehensions ---")

# Dict comprehension
squares = {x: x**2 for x in range(6)}
print(f"Dict comp: {squares}")

# Set comprehension
unique_lengths = {len(word) for word in ["hello", "world", "hi", "hey"]}
print(f"Set comp: {unique_lengths}")

# Nested comprehension (matrix transpose)
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
transpose = [[row[i] for row in matrix] for i in range(3)]
print(f"Transpose: {transpose}")

In [ ]:
# ============================================
# GOTCHA 9: Unpacking tricks
# ============================================
print("--- Unpacking ---")

# Swap without temp
a, b = 1, 2
a, b = b, a
print(f"After swap: a={a}, b={b}")

# Star unpacking
first, *middle, last = [1, 2, 3, 4, 5]
print(f"first={first}, middle={middle}, last={last}")

# Ignore values
_, important, _ = ("skip", "keep", "skip")
print(f"important={important}")

In [ ]:
# ============================================
# GOTCHA 10: Truthy and Falsy values
# ============================================
print("--- Truthy / Falsy ---")

# Falsy values (everything else is truthy):
falsy_values = [False, None, 0, 0.0, 0j, "", [], (), {}, set(), frozenset()]
for val in falsy_values:
    assert not val, f"{val!r} should be falsy"
print(f"All falsy values confirmed: {[repr(v) for v in falsy_values]}")

# Practical use:
name = ""  # or name = None
display_name = name or "Anonymous"  # Short-circuit evaluation
print(f"Display: {display_name}")

---

# Summary — Interview Cheat Sheet

| Topic | Key Takeaway |
|-------|-------------|
| Mutable vs Immutable | Lists/dicts are mutable; strings/tuples/ints are immutable |
| `is` vs `==` | `is` checks identity (same object), `==` checks value equality |
| Shallow vs Deep copy | Shallow copies share nested objects; deep copies don't |
| GIL | Prevents true parallelism for CPU-bound threads in CPython |
| `*args/**kwargs` | Collect extra positional/keyword arguments |
| LEGB | Local → Enclosing → Global → Built-in scope resolution |
| Closures | Functions that remember enclosing scope variables |
| Decorators | Functions that wrap other functions; use `@functools.wraps` |
| Mutable defaults | `def f(x=[])` is a bug; use `None` sentinel instead |
| MRO | C3 linearization: children before parents, left before right |
| `__repr__` vs `__str__` | `repr` for developers, `str` for users |
| Generators | Use `yield` for lazy, memory-efficient iteration |
| Context managers | `with` statement for guaranteed setup/cleanup |
| Type hints | Annotations for static analysis, NOT enforced at runtime |
| Memory | Reference counting + cyclic garbage collector |

**Remember**: Interviewers care more about the **"why"** than the **"what"**. Explain the reasoning behind Python's design choices, not just syntax.